In [11]:
import os
import pandas as pd
import librosa 
import numpy as np
# import seaborn as sns
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import IPython.display as idp
from tensorflow.keras.models import load_model
import cv2
import warnings
warnings.simplefilter("ignore")
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [12]:
df=pd.read_csv(r"C:\Users\Rashiqua Munshi\Downloads\ProjectSchool\UrbanSound8K.csv")
df.head(10)

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.000000,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.500000,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.500000,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.000000,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.500000,72.500000,1,5,2,children_playing
5,100263-2-0-143.wav,100263,71.500000,75.500000,1,5,2,children_playing
6,100263-2-0-161.wav,100263,80.500000,84.500000,1,5,2,children_playing
7,100263-2-0-3.wav,100263,1.500000,5.500000,1,5,2,children_playing
8,100263-2-0-36.wav,100263,18.000000,22.000000,1,5,2,children_playing
9,100648-1-0-0.wav,100648,4.823402,5.471927,2,10,1,car_horn


In [13]:
df['class'].value_counts()

class
dog_bark            1000
children_playing    1000
air_conditioner     1000
street_music        1000
engine_idling       1000
jackhammer          1000
drilling            1000
siren                929
car_horn             429
gun_shot             374
Name: count, dtype: int64

In [14]:
audio_dataset_path=r"C:\Users\Rashiqua Munshi\Downloads\ProjectSchool\archive"
audio_dataset_path

'C:\\Users\\Rashiqua Munshi\\Downloads\\ProjectSchool\\archive'

In [15]:
X = []
y = []
up_width = 173
up_height = 40
for data in df.iterrows():
    file_name = os.path.join(os.path.abspath(audio_dataset_path),'fold'+str(data[1][5])+'/',str(data[1][0]))
    raw , sr = librosa.load(file_name)
    X_ = librosa.feature.mfcc(y=raw, sr=sr, n_mfcc=40)
    up_points = (up_width, up_height)
    X_ = cv2.resize(X_, up_points, interpolation= cv2.INTER_LINEAR)
    X.append(X_)
    y.append(data[1][7])

In [16]:
X = np.array(X)
y = np.array(y)

In [17]:
print(X.shape)
print(y.shape)

(8732, 40, 173)
(8732,)


In [18]:
y = to_categorical(le.fit_transform(y))
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)

In [19]:
X_train , X_test , y_train , y_test = train_test_split(X , y ,test_size=0.2, random_state=42)

In [20]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.15, random_state=42)

In [21]:
INPUTSHAPE = (up_height,up_width,1)

In [22]:
model =  models.Sequential([
    
                          layers.Conv2D(32 , (3,3),activation = 'relu',padding='valid', input_shape = INPUTSHAPE),  
                          layers.MaxPooling2D(2, padding='same'),
                          layers.Conv2D(128, (3,3), activation='relu',padding='valid'),
                          layers.MaxPooling2D(2, padding='same'),
                          layers.Dropout(0.3),
                          layers.Conv2D(128, (3,3), activation='relu',padding='valid'),
                          layers.MaxPooling2D(2, padding='same'),
                          layers.Dropout(0.3),
                          layers.GlobalAveragePooling2D(),
                          layers.Dense(256 , activation = 'relu'),
                          layers.Dense(10 , activation = 'softmax')
])
opt = keras.optimizers.Adam(learning_rate = 0.001)
model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = 'acc')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 38, 171, 32)       320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 19, 86, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 17, 84, 128)       36992     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 9, 42, 128)       0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 9, 42, 128)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 40, 128)        1

In [23]:
from datetime import datetime 
batch_size = 8
callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=8, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False)

start = datetime.now()
history = model.fit(X_train,y_train ,
            validation_data=(X_valid,y_valid),
            epochs=15,
            callbacks = [callback],batch_size=batch_size)
duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/15
743/743 [==============================] - 36s 47ms/step - loss: 1.5921 - acc: 0.4325 - val_loss: 1.2738 - val_acc: 0.5563
Epoch 2/15
743/743 [==============================] - 36s 49ms/step - loss: 1.0010 - acc: 0.6523 - val_loss: 1.1206 - val_acc: 0.6050
Epoch 3/15
743/743 [==============================] - 39s 52ms/step - loss: 0.7772 - acc: 0.7325 - val_loss: 0.8421 - val_acc: 0.7252
Epoch 4/15
743/743 [==============================] - 39s 53ms/step - loss: 0.6286 - acc: 0.7901 - val_loss: 0.6588 - val_acc: 0.7920
Epoch 5/15
743/743 [==============================] - 38s 51ms/step - loss: 0.5142 - acc: 0.8245 - val_loss: 0.5595 - val_acc: 0.8149
Epoch 6/15
743/743 [==============================] - 39s 53ms/step - loss: 0.4432 - acc: 0.8509 - val_loss: 0.4693 - val_acc: 0.8464
Epoch 7/15
743/743 [==============================] - 39s 52ms/step - loss: 0.3738 - acc: 0.8806 - val_loss: 0.5643 - val_acc: 0.8254
Epoch 8/15
743/743 [==============================] - 39s 53ms

In [28]:
Y_pred = model.predict(X_test)

55/55 [==============================] - 3s 46ms/step
